In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import logging
logger = logging.getLogger(__name__)

STATES = ["S", "I", "R"]

In [44]:
def get_dummies(states):
    probas = np.zeros(states.shape + (3,))
    for s in [0,1,2]:
        probas[:,:,s] = (states==s)*1
    assert np.all(probas.argmax(axis=2) == states)
    return probas


def get_infection_probas(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    """
    infected = (states == 1)
    N = len(states)
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            rate for i0, j, rate in transmissions
            if i0 == i and infected[j]
        ])
        infection_probas[i] = 1 - np.prod(1 - rates)
    return infection_probas


def propagate(current_states, infection_probas, recover_probas):
    """
    - current_states[i] = state of i
    - infection_probas[i]  = proba that i get infected (if susceptible)
    - recover_probas[i] = proba that i recovers (if infected)
    """
    next_states = np.zeros_like(current_states)
    for i, state in enumerate(current_states):
        if (state == 0):
            infected = np.random.rand() < infection_probas[i]
            next_states[i] = 1 if infected else 0
        elif (state == 1):
            recovered = np.random.rand() < recover_probas[i]
            next_states[i] = 2 if recovered else 1
        else:
            next_states[i] = 2
    return next_states


class EpidemicModel():
    def __init__(self, initial_states, x_pos, y_pos):
        assert len(x_pos) == len(y_pos) == len(initial_states)
        self.N = len(initial_states)
        self.initial_states = initial_states
        self.x_pos = x_pos
        self.y_pos = y_pos

    def time_evolution(self, recover_probas, transmissions, print_every=10):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - states[t, i] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        states = np.empty((T + 1, self.N))
        states[:] = np.nan
        states[0] = self.initial_states
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(states[t], transmissions[t])
            states[t+1] = propagate(states[t], infection_probas, recover_probas)
        self.states = states
        self.probas = get_dummies(states)

    def plot(self, t):
        fig, ax = plt.subplots(1, 1, figsize = (5,5))
        for idx, state in enumerate(STATES):
            ind = np.where(self.states[t] == idx)
            ax.scatter(self.x_pos[ind], self.y_pos[ind], label=state)
        ax.set(title="t = %d" % t)
        ax.legend()

    def get_counts(self):
        counts = {
            state: (self.states == idx).sum(axis=1)
            for idx, state in enumerate(STATES)
        }
        return pd.DataFrame(counts)

class ProximityModel(EpidemicModel):
    """
    Model:
    - N = population
    - mu = constant recovery proba
    - lamd = constant transmission rate (if in contact)
    - proba_contact = np.exp(-distance / scale)
    - initial_states = random patient zero
    - x_pos, y_pos = random uniform

    You can also provide the initial_states, x_pos, y_pos or proba_contact.
    """
    def __init__(self, N, scale, mu, lamb,
    initial_states = None, x_pos = None, y_pos = None, proba_contact = None):
        self.scale = scale
        self.mu = mu
        self.lamb = lamb
        # initial states : patient zero infected
        if initial_states is None:
            patient_zero = np.random.randint(N)
            initial_states = np.zeros(N)
            initial_states[patient_zero] = 1
        # positions
        x_pos = np.sqrt(N)*np.random.rand(N) if x_pos is None else x_pos
        y_pos = np.sqrt(N)*np.random.rand(N) if y_pos is None else y_pos
        if proba_contact is None:
            # proba of contact = np.exp(-distance / scale)
            pos = np.array([x_pos, y_pos]).T
            assert pos.shape == (N, 2)
            distance = squareform(pdist(pos))
            proba_contact = np.exp(-distance / scale)
            np.fill_diagonal(proba_contact, False) # no contact with oneself
        self.proba_contact = proba_contact
        # expected number of contacts
        self.n_contacts = proba_contact.sum()/N
        # constant recovery proba
        self.recover_probas = mu*np.ones(N)
        super().__init__(initial_states, x_pos, y_pos)

    def sample_contacts(self):
        "contacts[i,j] = if i and j in contact"
        contacts = np.random.rand(self.N, self.N) < self.proba_contact
        return contacts

    def sample_transmissions(self):
        "transmissions = list of t, i, j, lambda_ij"
        contacts = self.sample_contacts()
        print("contacts" , contacts)
        i, j = np.where(contacts)
        print("i:", i,"j:", j)
        # constant rate = lamb
        rates = self.lamb * np.ones(len(i))
        print("rates", rates)
        transmissions = list(zip(i, j, rates))
        print("transmissions", transmissions)
        # sanity check
        assert contacts.sum() == len(transmissions)
        assert np.all(i != j)
        return transmissions

    def generate_transmissions(self, T):
        self.transmissions = [self.sample_transmissions() for t in range(T)]

    def run(self, T, print_every=10):
        print("Generating transmissions")
        self.generate_transmissions(T)
        print("Running simulation")
        self.time_evolution(
            self.recover_probas, self.transmissions, print_every=print_every
        )



In [45]:
model = ProximityModel(N=10, scale=1.2, mu=0.01, lamb=0.02)
print("expected number of contacts %.1f" % model.n_contacts)

expected number of contacts 2.6


In [ ]:
model.run(500, print_every=100)

In [108]:
a = [{'infectiousness': 0, 'other_human_id': 62, 'human_id':99, 'timestep':1, 'i_infected': True, 'j_infected': False},{'infectiousness': 0, 'other_human_id': 62, 'human_id':99, 'timestep':1, 'i_infected': False, 'j_infected': False}]

In [ ]:
def get

In [ ]:
def time_evolution(self, recover_probas, transmissions, print_every=10):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - states[t, i] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        states = np.empty((T + 1, self.N))
        states[:] = np.nan
        states[0] = self.initial_states
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(states[t], transmissions[t])
            states[t+1] = propagate(states[t], infection_probas, recover_probas)
        self.states = states
        self.probas = get_dummies(states)

In [ ]:
model.run(500, print_every=100)

In [82]:
import pickle
data = pickle.load(open("./data.pkl", 'rb'))

In [83]:
import datetime

In [104]:
def time_in_range(start, end, xs,xe):
    #Return true if x is in the range [start, end]
    if start <= end:
        return start<= xs and xe <= end
    else:
        return start<= xs or xe <= end



def time_slice(simulationls):
    # Observing nodes for 1 day over time-granuality(1 hour)
    # The date-time of encounter + duration should fall in the time slice
    time_slice_start = datetime.datetime(2020,3,16,0,0,0)
    time_slice_end = datetime.datetime(2020,3,17,0,0,0)
    
    dlist=[]
    interactions={}
    while(time_slice_start <= time_slice_end):
        print(time_slice_start)
        for d in simulationls:
            try: 
                #The nature of encounter should be "encounter" 
                # The date-time of enco'unter + duration should fall in the time slice
                time_interact_start = d['time']
                time_interact_end = time_interact_start + datetime.timedelta(seconds=d['payload']['observed']['duration'])

                if(time_in_range(time_slice_start,time_slice_start+datetime.timedelta(hours=1) ,time_interact_start,time_interact_end)):

                        interactions['infectiousness']=d['payload']['unobserved']['human1']['infectiousness']
                        interactions['human_id']=d['human_id']
                        interactions['other_human_id']=d['payload']['unobserved']['human2_id']
                        interactions['datetime']=d['time']
                        dlist.append(interactions)
                        print(dlist)
                time_slice_start+=datetime.timedelta(hours=1)
            except:
                pass

In [103]:
time_slice(data)

2020-03-16 00:00:00


In [57]:
data['payload']['observed']['location_type']['duration']

TypeError: list indices must be integers or slices, not str

In [107]:
a = [{'infectiousness': 0, 'other_human_id': 62, 'human_id':99, 'timestep':1, 'i_infected': True, 'j_infected': False},{'infectiousness': 0, 'other_human_id': 62, 'human_id':99, 'timestep':1, 'i_infected': False, 'j_infected': False}]

In [119]:
import pickle

In [120]:
data = pickle.load(open("./data.pkl",'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './data.pkl'